In [4]:
import torch

In [5]:
#dataset

from sklearn.datasets import fetch_california_housing

In [6]:
#py
import torch.nn as nn
from torch.utils.data import TensorDataset , DataLoader
import torchmetrics

In [7]:
#other libraries 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [8]:
housing = fetch_california_housing()

In [9]:
print(housing.data.shape , housing.target.shape)

(20640, 8) (20640,)


In [10]:
#lets scale

scale_data = StandardScaler()
housing_data = scale_data.fit_transform(housing.data)

scale_target = StandardScaler()
housing_target= scale_target.fit_transform(housing.target.reshape(-1 , 1)).flatten()

In [11]:
print(housing_data.shape , housing_target.shape)

(20640, 8) (20640,)


In [12]:
#data splittinh

x_train_full , x_test , y_train_full , y_test = train_test_split(housing_data , housing_target ,
                                                                  random_state=42 , test_size=0.15)

x_train , x_valid , y_train , y_valid = train_test_split(x_train_full , y_train_full ,
                                                         random_state=42 , test_size=0.15)

In [13]:
#convert to tensors 

x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
x_valid = torch.FloatTensor(x_valid)

y_train = torch.FloatTensor(y_train).reshape(-1  ,1 )
y_test = torch.FloatTensor(y_test).reshape(-1  ,1 )
y_valid = torch.FloatTensor(y_valid).reshape(-1  ,1 )





In [14]:
#random initialization

n_features = x_train.shape[1]
torch.manual_seed(42)
w = torch.rand((n_features , 1) , requires_grad=True)
b = torch.tensor(0. , requires_grad=True)

In [15]:
#low level api batch gradient descent
learning_rate = 0.4
n_epoch = 20

for epoch in range(n_epoch):
    #forward pass
    y_pred = x_train @ w + b
    loss = ((y_pred - y_train)**2).mean()
    #backward pass
    loss.backward()
    
    #update
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        b.grad.zero_()
        w.grad.zero_()
        
    print(f"Epoch number : {epoch + 1}/{n_epoch}",
          f"loss: {loss.item()}")
        
    

Epoch number : 1/20 loss: 3.6242189407348633
Epoch number : 2/20 loss: 1.5359041690826416
Epoch number : 3/20 loss: 1.0670270919799805
Epoch number : 4/20 loss: 0.8353963494300842
Epoch number : 5/20 loss: 0.7038992047309875
Epoch number : 6/20 loss: 0.6227049827575684
Epoch number : 7/20 loss: 0.5698260068893433
Epoch number : 8/20 loss: 0.5339987277984619
Epoch number : 9/20 loss: 0.5088570713996887
Epoch number : 10/20 loss: 0.4905947148799896
Epoch number : 11/20 loss: 0.47686341404914856
Epoch number : 12/20 loss: 0.466187447309494
Epoch number : 13/20 loss: 0.45762598514556885
Epoch number : 14/20 loss: 0.4505714178085327
Epoch number : 15/20 loss: 0.44462496042251587
Epoch number : 16/20 loss: 0.4395205080509186
Epoch number : 17/20 loss: 0.4350760281085968
Epoch number : 18/20 loss: 0.4311641454696655
Epoch number : 19/20 loss: 0.427692711353302
Epoch number : 20/20 loss: 0.42459341883659363


In [16]:
#high level api 

def train_bgd(model , optimizer , criterion , x_train , y_train , n_epoch):
    for epoch in range(n_epoch):
        #forward pass 
        y_pred = model(x_train)
        loss = criterion(y_pred , y_train)
        
        #backward 
        loss.backward()
        
        #optimize
        
        optimizer.step()
        optimizer.zero_grad()
        
        print(f"Epoch Number : {epoch+1}/{n_epoch}",
              f"Loss : {loss.item()}")
        
torch.manual_seed(42)
model = nn.Linear(in_features=n_features , out_features=1)
optimizer = torch.optim.SGD(model.parameters() , lr=learning_rate)
mse = nn.MSELoss()
train_bgd(model , optimizer , mse , x_train , y_train , n_epoch)

Epoch Number : 1/20 Loss : 1.0317535400390625
Epoch Number : 2/20 Loss : 0.4976767301559448
Epoch Number : 3/20 Loss : 0.4637906849384308
Epoch Number : 4/20 Loss : 0.45154303312301636
Epoch Number : 5/20 Loss : 0.4438009560108185
Epoch Number : 6/20 Loss : 0.4378271698951721
Epoch Number : 7/20 Loss : 0.4328160285949707
Epoch Number : 8/20 Loss : 0.4284723699092865
Epoch Number : 9/20 Loss : 0.4246593713760376
Epoch Number : 10/20 Loss : 0.4212949275970459
Epoch Number : 11/20 Loss : 0.4183187484741211
Epoch Number : 12/20 Loss : 0.41568195819854736
Epoch Number : 13/20 Loss : 0.4133431017398834
Epoch Number : 14/20 Loss : 0.4112664759159088
Epoch Number : 15/20 Loss : 0.409420907497406
Epoch Number : 16/20 Loss : 0.40777918696403503
Epoch Number : 17/20 Loss : 0.40631744265556335
Epoch Number : 18/20 Loss : 0.40501484274864197
Epoch Number : 19/20 Loss : 0.40385299921035767
Epoch Number : 20/20 Loss : 0.40281566977500916


In [17]:
#implementing reg mlp using mini batch gradient descent 
#with gpu acceleration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model , optimizer , criterion , train_loader , n_epoch):
    model = model.to(device)
    model.train()
    
    for epoch in range(n_epoch):
        total_loss = 0
        for x_batch , y_batch in train_loader:
            #on gpu
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            #forward pass
            y_pred = model(x_batch)
            
            #loss
            loss = criterion(y_pred , y_batch)
            total_loss += loss.item()
            
            #backward
            loss.backward()
            
            #optimizer
            
            optimizer.step()
            optimizer.zero_grad()
            
        mean_loss = total_loss/len(train_loader)
        print(f"Epoch : {epoch+1}/{n_epoch}",
              f"Loss : {mean_loss:.4f}")
        
#model mlp

learning_rate = 0.01
torch.manual_seed(42)

model = nn.Sequential(
    nn.Linear(n_features , 50),
    nn.ReLU(),
    nn.Linear(50 , 40),
    nn.ReLU(),
    nn.Linear(40 , 1)
)

optimizer = torch.optim.SGD(params= model.parameters() , lr=learning_rate)
mse = nn.MSELoss()

#dataset 
train_dataset = TensorDataset(x_train , y_train)
train_loader = DataLoader(train_dataset , batch_size=100 , shuffle=True ,
                          pin_memory=True)



train(model , optimizer , mse , train_loader , 20)       
            
            
            
            




Epoch : 1/20 Loss : 0.6747
Epoch : 2/20 Loss : 0.4486
Epoch : 3/20 Loss : 0.4165
Epoch : 4/20 Loss : 0.3928
Epoch : 5/20 Loss : 0.3744
Epoch : 6/20 Loss : 0.3613
Epoch : 7/20 Loss : 0.3507
Epoch : 8/20 Loss : 0.3406
Epoch : 9/20 Loss : 0.3410
Epoch : 10/20 Loss : 0.3276
Epoch : 11/20 Loss : 0.3218
Epoch : 12/20 Loss : 0.3167
Epoch : 13/20 Loss : 0.3114
Epoch : 14/20 Loss : 0.3091
Epoch : 15/20 Loss : 0.3059
Epoch : 16/20 Loss : 0.3004
Epoch : 17/20 Loss : 0.2978
Epoch : 18/20 Loss : 0.2938
Epoch : 19/20 Loss : 0.2907
Epoch : 20/20 Loss : 0.2874


In [21]:
#eval 

def evaluate(model , metric , data_loader):
    model.eval()
    metric.reset()
    with torch.no_grad():
        for x_batch , y_batch in data_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x_batch)
            metric.update(y_pred , y_batch)
    return metric.compute() 

In [22]:
#val dataset 

val_dataset = TensorDataset(x_valid , y_valid)
val_loader = DataLoader(val_dataset , batch_size = 100 , shuffle=True , pin_memory=True)

rmse = torchmetrics.MeanSquaredError(squared=False).to(device)
print(evaluate(model , rmse , val_loader))

tensor(0.5340, device='cuda:0')
